<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Preliminaries" data-toc-modified-id="Preliminaries-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Preliminaries</a></span><ul class="toc-item"><li><span><a href="#Define-filepaths" data-toc-modified-id="Define-filepaths-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Define filepaths</a></span></li><li><span><a href="#Load-country-list-and-mapping-dictionary" data-toc-modified-id="Load-country-list-and-mapping-dictionary-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Load country list and mapping dictionary</a></span></li><li><span><a href="#Read-data-dictionary" data-toc-modified-id="Read-data-dictionary-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Read data dictionary</a></span></li></ul></li><li><span><a href="#Staging-(pre-processing)-to-create-exceptional-indicators´-raw-data" data-toc-modified-id="Staging-(pre-processing)-to-create-exceptional-indicators´-raw-data-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Staging (pre-processing) to create exceptional indicators´ raw data</a></span></li><li><span><a href="#Extract---Transform---Load-Loop" data-toc-modified-id="Extract---Transform---Load-Loop-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Extract - Transform - Load Loop</a></span><ul class="toc-item"><li><span><a href="#API-sources" data-toc-modified-id="API-sources-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>API sources</a></span><ul class="toc-item"><li><span><a href="#CSV-API-sources" data-toc-modified-id="CSV-API-sources-3.1.1"><span class="toc-item-num">3.1.1&nbsp;&nbsp;</span>CSV API sources</a></span></li><li><span><a href="#JSON-API-sources" data-toc-modified-id="JSON-API-sources-3.1.2"><span class="toc-item-num">3.1.2&nbsp;&nbsp;</span>JSON API sources</a></span></li></ul></li><li><span><a href="#HTML-Sources" data-toc-modified-id="HTML-Sources-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>HTML Sources</a></span><ul class="toc-item"><li><span><a href="#UN-Treaty-Sources" data-toc-modified-id="UN-Treaty-Sources-3.2.1"><span class="toc-item-num">3.2.1&nbsp;&nbsp;</span>UN Treaty Sources</a></span></li></ul></li><li><span><a href="#Export-concatented-dataframe" data-toc-modified-id="Export-concatented-dataframe-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Export concatented dataframe</a></span></li></ul></li><li><span><a href="#DEVELOPMENT-AND-TRASH-AREA" data-toc-modified-id="DEVELOPMENT-AND-TRASH-AREA-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>DEVELOPMENT AND TRASH AREA</a></span><ul class="toc-item"><li><span><a href="#Extract-selenium-sources--->-This-code-is-stable-as-of-06.11.20,-TO-DO-is-to-put-this-into-a-loop-(which-must-be-done-in-container,-so-I-can-only-do-it-once-James-has-looked-at-the-issue-with-Chrome-driver)" data-toc-modified-id="Extract-selenium-sources--->-This-code-is-stable-as-of-06.11.20,-TO-DO-is-to-put-this-into-a-loop-(which-must-be-done-in-container,-so-I-can-only-do-it-once-James-has-looked-at-the-issue-with-Chrome-driver)-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Extract selenium sources --&gt; This code is stable as of 06.11.20, TO DO is to put this into a loop (which must be done in container, so I can only do it once James has looked at the issue with Chrome driver)</a></span></li><li><span><a href="#Extract-the-countries" data-toc-modified-id="Extract-the-countries-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Extract the countries</a></span><ul class="toc-item"><li><span><a href="#Extract-UN-Treaty-data" data-toc-modified-id="Extract-UN-Treaty-data-4.2.1"><span class="toc-item-num">4.2.1&nbsp;&nbsp;</span>Extract UN Treaty data</a></span></li></ul></li></ul></li></ul></div>

# Preliminaries

In [ ]:
# Required standard libraries
import pandas as pd
import json
import urllib
import requests
import os
import re
import numpy as np
import bs4 as bs
import selenium
import html5lib
# import nltk
import datetime
from selenium import webdriver

# Extractors 
import extract

# Cleansers (cluster specific)
import cleanse

# Normalizer (generalised across all clusters)
from normalize import scaler

# Utils
# from utils import utils

## Define filepaths

In [2]:
# Define the export path for all data exports
from pathlib import Path

# Current working directory
cwd = Path('.')

# Folder with data-in artifacts, quired to run this script
data_in = cwd / 'data_in'

# Folder to export raw data
data_sources_raw = cwd / 'data_out' / 'data_raw'
data_sources_raw.mkdir(parents=True, exist_ok=True)

# Folder to export cleansed data
data_sources_cleansed = cwd / 'data_out' / 'data_cleansed'
data_sources_cleansed.mkdir(parents=True, exist_ok=True)

# Folder to export normalized data
data_sources_normalized = cwd / 'data_out' / 'data_normalized'
data_sources_normalized.mkdir(parents=True, exist_ok=True)

## Load country list and mapping dictionary

In [3]:
# Load the list of countries which contains all different variations of country names 
country_full_list = pd.read_excel(
    data_in / 'all_countrynames_list.xlsx',
    keep_default_na = False).drop_duplicates()

# Create a version of the list with unique ISO2 and ISO3 codes
country_iso_list = country_full_list.drop_duplicates(subset = 'COUNTRY_ISO_2')

# Country CRBA list, this is the list of the countries that should be in the final CRBA indicator list
country_crba_list = pd.read_excel(
    data_in / 'crba_country_list.xlsx',
    header = None,
    usecols = [0, 1], 
    names = ['COUNTRY_ISO_3', 'COUNTRY_NAME']).merge(
        right = country_iso_list[['COUNTRY_ISO_2', 'COUNTRY_ISO_3']],
        how = 'left',
        on='COUNTRY_ISO_3',
        validate = 'one_to_one')

# Run the column mapper script to load the mapping dictionary
with open(data_in / 'column_mapping.py') as file:
    exec(file.read())

# Run the column mapper script to load the mapping dictionary
with open(data_in / 'value_mapping.py') as file:
    exec(file.read())

## Read data dictionary

In [ ]:
# sources sheet
crba_data_dictionary_source = pd.read_excel(
    data_in / 'indicator_dictionary_CRBA.xlsx',
    sheet_name = "Source",
    keep_default_na = False
)

# snapshot sheet
crba_data_dictionary_snapshot = pd.read_excel(
    data_in / 'indicator_dictionary_CRBA.xlsx',
    sheet_name = "Snapshot",
    keep_default_na = False
)

# indicator sheet
crba_data_dictionary_indicator = pd.read_excel(
    data_in / 'indicator_dictionary_CRBA.xlsx',
    sheet_name = "Indicator",
    keep_default_na = False
)

# Input lists
crba_data_dictionary_input_list = pd.read_excel(
    data_in / 'indicator_dictionary_CRBA.xlsx',
    sheet_name = "Input_Lists",
    keep_default_na = False
)

# Add 2-digit shortcodes of index, issue and category to indicators sheet
crba_data_dictionary_indicator = crba_data_dictionary_indicator.merge(
    right=crba_data_dictionary_input_list[['INDEX', 'INDEX_CODE']],
    left_on='INDEX',
    right_on='INDEX'
).merge(
    right=crba_data_dictionary_input_list[['ISSUE', 'ISSUE_CODE']],
    left_on='ISSUE',
    right_on='ISSUE'
).merge(
    right=crba_data_dictionary_input_list[['CATEGORY', 'CATEGORY_CODE']],
    left_on='CATEGORY',
    right_on='CATEGORY'
)

# Create indicator code prefix (INDEX-ISSUE_CAEGORY CODE)
crba_data_dictionary_indicator = crba_data_dictionary_indicator.assign(
    INDICATOR_CODE_PREFIX = crba_data_dictionary_indicator.INDEX_CODE +
    "_" +
    crba_data_dictionary_indicator.ISSUE_CODE+
    "_"+
    crba_data_dictionary_indicator.CATEGORY_CODE+
    "_")

# Create indicator code
crba_data_dictionary_indicator = crba_data_dictionary_indicator.assign(
    INDICATOR_CODE = crba_data_dictionary_indicator.INDICATOR_CODE_PREFIX + crba_data_dictionary_indicator.INDICATOR_NAME.apply(
    lambda x: utils.create_ind_code(x)
))

In [ ]:
import importlib, inspect

extractors = { 
    cls.type: cls for name, cls in inspect.getmembers(
        importlib.import_module("extract"), 
        inspect.isclass
    ) if hasattr(cls, 'type')
}

# Staging (pre-processing) to create exceptional indicators´ raw data 

#

In [ ]:
# Pre-process S-180 and S-181
# Important: File requires having filepaths from above defined and pandas already imported

with open(data_in / 'staging_create_raw_data.py') as file:
    exec(file.read())

# Extract - Transform - Load Loop
## API sources
### CSV API sources

In [ ]:
# CSV sources
api_sources = crba_data_dictionary_source[
    (crba_data_dictionary_source["SOURCE_TYPE"] == "API (ILO)") | 
    (crba_data_dictionary_source["SOURCE_TYPE"] == "API (UNESCO)") | 
    (crba_data_dictionary_source["SOURCE_TYPE"] == "API (WHO)") | 
    (crba_data_dictionary_source["SOURCE_TYPE"] == "API (UNICEF)")
].merge(
    right = crba_data_dictionary_snapshot,
    on = "SOURCE_ID"
).merge(
    right = crba_data_dictionary_indicator,
    on = 'INDICATOR_ID'
)

# define emty dataframe
combined_cleansed_csv = pd.DataFrame()
combined_normalized_csv = pd.DataFrame()

# Loop to extract data from API sources
for index, row in api_sources.iterrows():
    # Log
    print("\n - - - - - \n Extracting source {} \n".format(row["SOURCE_ID"]))
    
    # Extraction section
    try:
        # Extract data
        dataframe = extract.CSVExtractor.extract(url = row["ENDPOINT_URL"])
        
        # Save raw data
        dataframe.to_csv(
            data_sources_raw / str(row["SOURCE_ID"] + "_raw.csv"),
            sep = ";"
            )
    
    except:
       print("There was a problem with extraction of source {} \n".format(row["SOURCE_ID"]))
    
    
    # Log that we are entering cleasning
    print("\n - - - - - \n Cleansing source {} \n".format(row["SOURCE_ID"]))
    
    # Cleansing
    dataframe = cleanse.Cleanser().extract_who_raw_data(
        raw_data=dataframe,
        variable_type = row["VALUE_LABELS"],
        display_value_col="Display Value"
    )
    
    dataframe = cleanse.Cleanser().rename_and_discard_columns(
        raw_data=dataframe,
        mapping_dictionary=mapping_dict,
        final_sdmx_col_list=sdmx_df_columns_all
    )

    dataframe = cleanse.Cleanser().retrieve_latest_observation(
        renamed_data=dataframe,
        dim_cols = sdmx_df_columns_dims,
        country_cols = sdmx_df_columns_country,
        time_cols = sdmx_df_columns_time,
        attr_cols=sdmx_df_columns_attr,
    )

    dataframe = cleanse.Cleanser().add_and_discard_countries(
        grouped_data=dataframe,
        crba_country_list=country_crba_list,
        country_list_full = country_full_list
    )

    dataframe = cleanse.Cleanser().add_cols_fill_cells(
        grouped_data_iso_filt=dataframe,
        dim_cols=sdmx_df_columns_dims,
        time_cols=sdmx_df_columns_time,
        indicator_name_string=row["INDICATOR_NAME_x"],
        index_name_string=row["INDEX"],
        issue_name_string=row["ISSUE"],
        category_name_string=row["CATEGORY"],
        indicator_code_string=row["INDICATOR_CODE"],
        indicator_source_string=row["ADDRESS"],
        indicator_source_body_string=row["SOURCE_BODY"],
        indicator_description_string=row["INDICATOR_DESCRIPTION"],
        indicator_explanation_string=row["INDICATOR_EXPLANATION"],
        indicator_data_extraction_methodology_string=row["EXTRACTION_METHODOLOGY"],
        source_title_string=row["SOURCE_TITLE"],
        source_api_link_string=row["ENDPOINT_URL"]
    )

    dataframe = cleanse.Cleanser().map_values(
        cleansed_data = dataframe,
        value_mapping_dict = value_mapper
    )
    
    dataframe_cleansed = cleanse.Cleanser().encode_categorical_variables(
        dataframe = dataframe,
        encoding_string = row["VALUE_LABELS"]
    )

    cleanse.Cleanser().create_log_report(
        cleansed_data=dataframe_cleansed
    )

    # Append dataframe to combined dataframe
    combined_cleansed_csv = combined_cleansed_csv.append(
        other = dataframe_cleansed
    )

    # Save cleansed data
    dataframe_cleansed.to_csv(
        data_sources_cleansed / str(row["SOURCE_ID"] + "_cleansed.csv"),
        sep = ";")
    
    # Normalizing
    dataframe_normalized = scaler.normalizer(
        cleansed_data = dataframe_cleansed,
        sql_subset_query_string=row["DIMENSION_VALUES_NORMALIZATION"],
        # dim_cols=sdmx_df_columns_dims,
        variable_type = row["VALUE_LABELS"],
        is_inverted = row["INVERT_NORMALIZATION"],
        whisker_factor=1.5,
        raw_data_col="RAW_OBS_VALUE",
        scaled_data_col_name="SCALED_OBS_VALUE",
        maximum_score=10,
        )
    
    dataframe_normalized.to_csv(
        data_sources_normalized / str(row["SOURCE_ID"] + "_normalized.csv"),
        sep = ";")

    # Append dataframe to combined dataframe
    combined_normalized_csv = combined_normalized_csv.append(
        other = dataframe_normalized
    )

### JSON API sources

In [ ]:
# JSON sources
api_sources = crba_data_dictionary_source[
    (crba_data_dictionary_source["SOURCE_TYPE"] == "API (SDG)")
].merge(
    right = crba_data_dictionary_snapshot,
    on = "SOURCE_ID"
).merge(
    right = crba_data_dictionary_indicator,
    on = 'INDICATOR_ID'
)

# Loop to extract data from API sources
for index, row in api_sources.iterrows():
    # Log
    print("\n - - - - - \n Extracting source {} \n".format(row["SOURCE_ID"]))
    
    # Exraction section
    try:
        # Extract data 
        dataframe = extract.JSONExtractor.extract(url = row["ENDPOINT_URL"])
        
        # Save dataframe
        dataframe.to_csv(
            data_sources_raw / str(row["SOURCE_ID"] + "_raw.csv"),
            sep = ";")
    except:
        print("There was an issue with source {}".format(row["SOURCE_ID"]))
    
    # Log that we are entering cleasning
    print("\n - - - - - \n Cleansing source {} \n".format(row["SOURCE_ID"]))
    
    # Cleansing in 
    dataframe = cleanse.Cleanser().extract_who_raw_data(
        raw_data=dataframe,
        variable_type = row["VALUE_LABELS"],
        display_value_col="Display Value"
    )
    
    dataframe = cleanse.Cleanser().rename_and_discard_columns(
        raw_data=dataframe,
        mapping_dictionary=mapping_dict,
        final_sdmx_col_list=sdmx_df_columns_all
    )

    dataframe = cleanse.Cleanser().retrieve_latest_observation(
        renamed_data=dataframe,
        dim_cols = sdmx_df_columns_dims,
        country_cols = sdmx_df_columns_country,
        time_cols = sdmx_df_columns_time,
        attr_cols=sdmx_df_columns_attr,
    )

    dataframe = cleanse.Cleanser().add_and_discard_countries(
        grouped_data=dataframe,
        crba_country_list=country_crba_list,
        country_list_full = country_full_list
    )

    dataframe = cleanse.Cleanser().add_cols_fill_cells(
        grouped_data_iso_filt=dataframe,
        dim_cols=sdmx_df_columns_dims,
        time_cols=sdmx_df_columns_time,
        indicator_name_string=row["INDICATOR_NAME_x"],
        index_name_string=row["INDEX"],
        issue_name_string=row["ISSUE"],
        category_name_string=row["CATEGORY"],
        indicator_code_string=row["INDICATOR_CODE"],
        indicator_source_string=row["ADDRESS"],
        indicator_source_body_string=row["SOURCE_BODY"],
        indicator_description_string=row["INDICATOR_DESCRIPTION"],
        source_title_string=row["SOURCE_TITLE"],
        indicator_explanation_string=row["INDICATOR_EXPLANATION"],
        indicator_data_extraction_methodology_string=row["EXTRACTION_METHODOLOGY"],
        source_api_link_string=row["ENDPOINT_URL"]
    )

    dataframe = cleanse.Cleanser().map_values(
        cleansed_data = dataframe,
        value_mapping_dict = value_mapper
    )
    
    dataframe_cleansed = cleanse.Cleanser().encode_categorical_variables(
        dataframe = dataframe,
        encoding_string = row["VALUE_LABELS"]
    )

    cleanse.Cleanser().create_log_report(
        cleansed_data=dataframe_cleansed
    )

    # Append dataframe to combined dataframe
    combined_cleansed_csv = combined_cleansed_csv.append(
        other = dataframe_cleansed
    )

    # Save cleansed data
    dataframe_cleansed.to_csv(
        data_sources_cleansed / str(row["SOURCE_ID"] + "_cleansed.csv"),
        sep = ";")
    
    # Normalizing section
    dataframe_normalized = scaler.normalizer(
        cleansed_data = dataframe_cleansed,
        sql_subset_query_string=row["DIMENSION_VALUES_NORMALIZATION"],
        # dim_cols=sdmx_df_columns_dims,
        variable_type = row["VALUE_LABELS"],
        is_inverted = row["INVERT_NORMALIZATION"],
        whisker_factor=1.5,
        raw_data_col="RAW_OBS_VALUE",
        scaled_data_col_name="SCALED_OBS_VALUE",
        maximum_score=10,
        )
    
    dataframe_normalized.to_csv(
        data_sources_normalized / str(row["SOURCE_ID"] + "_normalized.csv"),
        sep = ";")

    # Append dataframe to combined dataframe
    combined_normalized_csv = combined_normalized_csv.append(
        other = dataframe_normalized
    )

## HTML Sources
### UN Treaty Sources

In [ ]:
# UN Treaty HTML sources
api_sources = crba_data_dictionary_source[
    (crba_data_dictionary_source["SOURCE_BODY"] == "UN Treaties")
].merge(
    right = crba_data_dictionary_snapshot,
    on = "SOURCE_ID"
).merge(
    right = crba_data_dictionary_indicator,
    on = 'INDICATOR_ID'
)

# Loop to extract data from API sources
for index, row in api_sources.iterrows():
    # Log
    print("\n - - - - - \n Extracting source {} \n".format(row["SOURCE_ID"]))
    
    # Exraction section
    dataframe = extract.HTMLExtractor().extract(url = row["ADDRESS"])
    
    # Save dataframe
    dataframe.to_csv(
        data_sources_raw / str(row["SOURCE_ID"] + "_raw.csv"),
        sep = ";")

    # Cleansing
    # Log that we are entering cleasning
    print("\n - - - - - \n Cleansing source {} \n".format(row["SOURCE_ID"]))
    
    # Cleansing
    dataframe = cleanse.Cleanser().rename_and_discard_columns(
        raw_data=dataframe,
        mapping_dictionary=mapping_dict,
        final_sdmx_col_list=sdmx_df_columns_all
    )

    dataframe = cleanse.Cleanser().add_and_discard_countries(
        grouped_data=dataframe,
        crba_country_list=country_crba_list,
        country_list_full = country_full_list
    )

    dataframe = cleanse.Cleanser().add_cols_fill_cells(
        grouped_data_iso_filt=dataframe,
        dim_cols=sdmx_df_columns_dims,
        time_cols=sdmx_df_columns_time,
        indicator_name_string=row["INDICATOR_NAME_x"],
        index_name_string=row["INDEX"],
        issue_name_string=row["ISSUE"],
        category_name_string=row["CATEGORY"],
        indicator_code_string=row["INDICATOR_CODE"],
        indicator_source_string=row["ADDRESS"],
        indicator_source_body_string=row["SOURCE_BODY"],
        indicator_description_string=row["INDICATOR_DESCRIPTION"],
        source_title_string=row["SOURCE_TITLE"],
        indicator_explanation_string=row["INDICATOR_EXPLANATION"],
        indicator_data_extraction_methodology_string=row["EXTRACTION_METHODOLOGY"],
        source_api_link_string=row["ENDPOINT_URL"]
    )

    dataframe_cleansed = cleanse.Cleanser().encode_ilo_un_treaty_data(
        dataframe = dataframe,
        treaty_source_body = row["SOURCE_BODY"]
    )

    cleanse.Cleanser().create_log_report(
        cleansed_data=dataframe_cleansed
    )

    # Append dataframe to combined dataframe
    combined_cleansed_csv = combined_cleansed_csv.append(
        other = dataframe_cleansed
    )

    # Save cleansed data
    dataframe_cleansed.to_csv(
        data_sources_cleansed / str(row["SOURCE_ID"] + "_cleansed.csv"),
        sep = ";")
    
    # Normalizing section
    dataframe_normalized = scaler.normalizer(
        cleansed_data = dataframe_cleansed,
        sql_subset_query_string=row["DIMENSION_VALUES_NORMALIZATION"],
        # dim_cols=sdmx_df_columns_dims,
        variable_type = row["VALUE_LABELS"],
        is_inverted = row["INVERT_NORMALIZATION"],
        whisker_factor=1.5,
        raw_data_col="RAW_OBS_VALUE",
        scaled_data_col_name="SCALED_OBS_VALUE",
        maximum_score=10,
        )
    
    dataframe_normalized.to_csv(
        data_sources_normalized / str(row["SOURCE_ID"] + "_normalized.csv"),
        sep = ";")

    # Append dataframe to combined dataframe
    combined_normalized_csv = combined_normalized_csv.append(
        other = dataframe_normalized
    )

## Export concatented dataframe

In [ ]:
# # # # # CLEANSED DATA

# Idenify all dimension columns in combined dataframe
available_dim_cols = []
for col in combined_cleansed_csv.columns:
    dim_col = re.findall("DIM_.+", col)
    # print(dim_col)
    if len(dim_col) == 1:
        available_dim_cols += dim_col

# Fill _T for all NA values of dimension columns
# 5b Fill in current year for time variable
combined_cleansed_csv[available_dim_cols] = combined_cleansed_csv[
    available_dim_cols
].fillna(value="_T")

# Export combined cleansed dataframe as a sample
combined_cleansed_csv.to_csv(
    path_or_buf = cwd / 'data_out' / 'combined_cleansed.csv',
    sep = ";"
)

# # # # # NORMALIZED DATA

# Idenify all dimension columns in combined dataframe
available_dim_cols = []
for col in combined_normalized_csv.columns:
    dim_col = re.findall("DIM_.+", col)
    # print(dim_col)
    if len(dim_col) == 1:
        available_dim_cols += dim_col

# Fill _T for all NA values of dimension columns
# 5b Fill in current year for time variable
combined_normalized_csv[available_dim_cols] = combined_normalized_csv[
    available_dim_cols
].fillna(value="_T")

# Export combined cleansed dataframe as a sample
combined_normalized_csv.to_csv(
    path_or_buf = cwd / 'data_out' / 'combined_normalized.csv',
    sep = ";"
)

# DEVELOPMENT AND TRASH AREA

In [ ]:
print(cwd)

In [ ]:
import requests
import bs4 as bs
import pandas as pd
import selenium
import os
from pathlib import Path
from selenium import webdriver

# cwd = Path('.')
cwd = os.getcwd()

# Current working directory
driver_location = cwd + '\\geckodriver.exe'

print(driver_location)

# Open the targete html. Must be done with selenium, because it doesnt work with normal URL request
#driver = webdriver.Firefox(executable_path="D:/Documents/2020/28_UNICEF/10_working_repo/data-etl/geckodriver.exe")
driver = webdriver.Firefox(executable_path=driver_location)

# Get HTTP response
response = driver.get("https://www.ilo.org/dyn/normlex/en/f?p=NORMLEXPUB:11300:0::NO:11300:P11300_INSTRUMENT_ID:312256:NO")


## Extract selenium sources --> This code is stable as of 06.11.20, TO DO is to put this into a loop (which must be done in container, so I can only do it once James has looked at the issue with Chrome driver)

In [4]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

# Specify location of chromedriver
cwd = os.getcwd()
driver_location = cwd + '\\chromedriver.exe'

# Add option to make it headless (so that it doesn't open an actual chrome window)
options = Options()
options.headless = True
driver = webdriver.Chrome(driver_location, chrome_options=options)

# Get HTTP response
# response = driver.get("https://www.ilo.org/dyn/normlex/en/f?p=NORMLEXPUB:11300:0::NO:11300:P11300_INSTRUMENT_ID:312256:NO")
# response = driver.get("https://www.ilo.org/dyn/normlex/en/f?p=NORMLEXPUB:11300:0::NO::P11300_INSTRUMENT_ID:312283")
# response = driver.get("https://www.ilo.org/dyn/normlex/en/f?p=NORMLEXPUB:11300:0::NO:11300:P11300_INSTRUMENT_ID:312328:NO")
response = driver.get("https://www.ilo.org/dyn/normlex/en/f?p=NORMLEXPUB:11300:0::NO:11300:P11300_INSTRUMENT_ID:312240:NO")

# Get response
# response = driver.get(html_url)

# Retrieve the actual html
html = driver.page_source

# Soupify
soup = bs.BeautifulSoup(html)

# Extract the target table as attribute
target_table = str(
    soup.find_all("table", {"cellspacing": "0", "class": "horizontalLine"})
)

# Create dataframe with the data
raw_data = pd.read_html(io=target_table, header=0)[
    0
]  # return is a list of DFs, specify [0] to get actual DF

# Cleansing
dataframe = cleanse.Cleanser().rename_and_discard_columns(
    raw_data=raw_data,
    mapping_dictionary=mapping_dict,
    final_sdmx_col_list=sdmx_df_columns_all
)

dataframe = cleanse.Cleanser().decompose_country_footnote_ilo_normlex(
    dataframe = dataframe,
    country_name_list = country_full_list.COUNTRY_NAME
)

dataframe = cleanse.Cleanser().add_and_discard_countries(
    grouped_data=dataframe,
    crba_country_list=country_crba_list,
    country_list_full = country_full_list
)

dataframe_cleansed = cleanse.Cleanser().encode_ilo_un_treaty_data(
    dataframe = dataframe,
    treaty_source_body='ILO NORMLEX'
)

# Normalizing section
dataframe_normalized = scaler.normalizer(
    cleansed_data = dataframe_cleansed,
    sql_subset_query_string=None
)

dataframe_normalized
# Code works - but still is missing step to map the countries
# dataframe

C:\Users\Michael\anaconda3\envs\unicef-test\lib\site-packages\ipykernel_launcher.py:32: DeprecationWarning: use options instead of chrome_options



 Calling function 'rename_and_discard_columns'...

 Calling function 'add_and_discard_countries'...

 Calling function 'add_cols_fill_cells'...


,COUNTRY_NAME,ATTR_TREATY_STATUS,ATTR_FOOTNOTE_OF_SOURCE,COUNTRY_ISO_2,COUNTRY_ISO_3,_merge,RAW_OBS_VALUE,ATTR_ENCODING_LABELS,SCALED_OBS_VALUE,OBS_STATUS
0,Afghanistan,In Force,,AF,AFG,both,2,"2=Yes, 1=No; as answer to the following questi...",10.0,nan
1,Albania,In Force,Excluding Article 11 by virtue of the ratifica...,AL,ALB,both,2,"2=Yes, 1=No; as answer to the following questi...",10.0,nan
2,Algeria,In Force,,DZ,DZA,both,2,"2=Yes, 1=No; as answer to the following questi...",10.0,nan
3,Argentina,In Force,,AR,ARG,both,2,"2=Yes, 1=No; as answer to the following questi...",10.0,nan
4,Armenia,In Force,Excluding Article 11 by virtue of the ratifica...,AM,ARM,both,2,"2=Yes, 1=No; as answer to the following questi...",10.0,nan
...,...,...,...,...,...,...,...,...,...,...
192,NaN,NaN,NaN,NaN,USA,right_only,1,"2=Yes, 1=No; as answer to the following questi...",0.0,nan
193,NaN,NaN,NaN,NaN,UZB,right_only,1,"2=Yes, 1=No; as answer to the following questi...",0.0,nan
194,NaN,NaN,NaN,NaN,VUT,right_only,1,"2=Yes, 1=No; as answer to the following questi...",0.0,nan
195,NaN,NaN,NaN,NaN,VNM,right_only,1,"2=Yes, 1=No; as answer to the following questi...",0.0,nan


In [ ]:
re.sub(dataframe["ATTR_FOOTNOTE_OF_SOURCE"][5]
       
# Speifically for ILO NORMLEX - extract country name if additonal info is given
#dataframe["ATTR_FOOTNOTE_OF_SOURCE"] = dataframe["COUNTRY_NAME"]
#dataframe["COUNTRY_NAME"] = dataframe["COUNTRY_NAME"].apply(extract_country_name)
#dataframe["ATTR_FOOTNOTE_OF_SOURCE"] = dataframe.apply(lambda x: re.sub(x['COUNTRY_NAME'], "", x["ATTR_FOOTNOTE_OF_SOURCE"]), 1)


## Extract the countries

In [ ]:
import bs4 as bs
import pandas
import os
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

# Specify location of chromedriver
cwd = os.getcwd()
driver_location = cwd + '\\chromedriver.exe'

# Add option to make it headless (so that it doesn't open an actual chrome window)
options = Options()
options.headless = True
driver = webdriver.Chrome(driver_location, chrome_options=options)

# Get HTTP response
# response = driver.get("https://www.ilo.org/dyn/normlex/en/f?p=NORMLEXPUB:11300:0::NO:11300:P11300_INSTRUMENT_ID:312256:NO")
response = driver.get("https://www.ilo.org/dyn/normlex/en/f?p=NORMLEXPUB:11300:0::NO::P11300_INSTRUMENT_ID:312283")

# Get response
# response = driver.get(html_url)

# Retrieve the actual html
html = driver.page_source

# Soupify
soup = bs.BeautifulSoup(html)

# Extract the target table as attribute
target_table = str(
    soup.find_all("table", {"cellspacing": "0", "class": "horizontalLine"})
)

# Create dataframe with the data
raw_data = pd.read_html(io=target_table, header=0)[
    0
]  # return is a list of DFs, specify [0] to get actual DF

# Cleansing
dataframe = cleanse.Cleanser().rename_and_discard_columns(
    raw_data=raw_data,
    mapping_dictionary=mapping_dict,
    final_sdmx_col_list=sdmx_df_columns_all
)

# dataframe

In [ ]:
dataframe

In [ ]:
print(country_full_list.loc[subset_list, "COUNTRY_NAME"].item())

In [ ]:
country_full_list.COUNTRY_NAME[47] in dataframe.COUNTRY_NAME[20]

In [ ]:
country_full_list.COUNTRY_NAME[47]

In [ ]:
# dataframe["COUNTRY_NAME"][20]

[re.search(x+'*?', dataframe.COUNTRY_NAME[20]) for x in country_full_list.COUNTRY_NAME]

In [ ]:
temp = extract_country_name(dataframe["COUNTRY_NAME"][20])

In [ ]:
temp.iloc[0]

In [ ]:
dataframe["COUNTRY_NAME"].apply(extract_country_name).sample(30)

In [ ]:
def extract_country_name(cell, country_name_list = country_full_list.COUNTRY_NAME):
    # Determine which country in the full country list is contained in string
    subset_list = [x in cell for x in country_name_list]
    
    # Sometimes several country names match, but we need exactly one
    if sum(subset_list)==0:
        print("No country name match")
    elif sum(subset_list)==1:
        # Retrieve the actual country name
        country_name = country_name_list[subset_list].item()
    else:
        # Retrieve the actual country name
        country_name = country_name_list[subset_list].iloc[0]
    
    return country_name

In [ ]:
country_full_list.COUNTRY_NAME[subset_list].item()

In [ ]:

country_full_list
    
subset_list = [x in dataframe.COUNTRY_NAME[2] for x in country_full_list.COUNTRY_NAME]

dataframe.COUNTRY_NAME[2] = country_full_list.loc[subset_list, "COUNTRY_NAME"].item()

dataframe

### Extract UN Treaty data

In [ ]:
import requests
import urllib
import bs4 as bs
import pandas as pd

raw_html_1 = requests.get("https://treaties.un.org/pages/ViewDetails.aspx?src=TREATY&mtdsg_no=XVIII-12-a&chapter=18&clang=_en")

# raw_html_1.text
raw_html_2 = urllib.request.urlopen("https://treaties.un.org/pages/ViewDetails.aspx?src=TREATY&mtdsg_no=XVIII-12-a&chapter=18&clang=_en")

print(raw_html_1)
print(raw_html_2)

soup = bs.BeautifulSoup(raw_html_1.text, features="lxml")
#soup_2 = bs.BeautifulSoup(raw_html_2, features="lxml")

#soup_2
# soup_1

# Extract the target table as attribute
target_table = str(
    soup.find_all(
        "table",
        {"class": "table table-striped table-bordered table-hover table-condensed"},
    )
)

# Create dataframe with the data
raw_data = pd.read_html(io=target_table, header=0)[
    0
]

raw_data

In [ ]:
# Identify duplicates

# Checking to see if there are any duplicate entries
duplicates = combined_normalized_csv[combined_normalized_csv.duplicated(
    subset = [
        "COUNTRY_ISO_3",
        "TIME_PERIOD",
        "COUNTRY_NAME",
        "COUNTRY_ISO_2",
        "RAW_OBS_VALUE",
        "DIM_SEX",
        "DIM_EDU_LEVEL",
        "DIM_AGE",
        "DIM_AGE_GROUP",
        "DIM_MANAGEMENT_LEVEL",
        "DIM_AREA_TYPE",
        "DIM_QUANTILE",
        "DIM_SDG_INDICATOR",
        "DIM_OCU_TYPE",
        "DIM_REP_TYPE",
        "DIM_SECTOR",
        "INDICATOR_CODE"
        ],
    keep = False

)]

duplicates.to_csv(
    path_or_buf = cwd / 'data_out' / 'duplicates.csv',
    sep = ";"
)